<a href="https://colab.research.google.com/github/tomknightatl/USCCB/blob/main/Build_Parishes_Database_From_Table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import required libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3

In [2]:
# Cell 1: Fetch the webpage content
url = "https://archatl.com/parishes/parish-directory/"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [7]:
# Cell 2: Extract the table data
table = soup.find('table', {'id': 'table_1'})
rows = table.find_all('tr')

data = []
for row in rows[1:]:  # Skip the header row
    cols = row.find_all('td')
    row_data = [col.text.strip() for col in cols[:-1]]  # Extract all columns except the last one

    # Extract the hyperlink from the last column
    web_col = cols[-1]
    link = web_col.find('a')
    if link:
        row_data.append(link.get('href'))
    else:
        row_data.append('')

    data.append(row_data)

In [8]:
# Cell 3: Create a DataFrame
columns = ['Name', 'Status', 'Deanery', 'EST', 'Street Address', 'City', 'State', 'Zipcode', 'Phone Number', 'Web']
df = pd.DataFrame(data, columns=columns)

In [9]:
# Cell 4: Create SQLite database and store the data
conn = sqlite3.connect('parishes.db')
df.to_sql('parishes', conn, if_exists='replace', index=False)

116

In [10]:
# Cell 5: Verify the data in the database
query = "SELECT * FROM parishes LIMIT 5"
result = pd.read_sql_query(query, conn)
print(result)

# Close the database connection
conn.close()

print("Data extraction and storage complete.")

                           Name           Status          Deanery   EST  \
0                    All Saints           Parish      North Metro  1977   
1         Capilla Santo Domingo  Pastoral Center        Northeast  2014   
2        Capilla San Juan Diego  Pastoral Center        Northwest         
3  Cathedral of Christ the King        Cathedral          Central  1936   
4               Christ Our Hope           Parish  Northeast Metro  1984   

            Street Address      City State Zipcode  Phone Number  \
0   2443 Mount Vernon Road  Dunwoody    GA   30338  770-393-3255   
1          427 Cash Street  Cornelia    GA   30531  706-754-4518   
2        1609 E Morris St.    Dalton    GA   30720  706-278-3107   
3  2699 Peachtree Road, NE   Atlanta    GA   30305  404-233-2145   
4       1786 Wellborn Road  Lithonia    GA   30058  770-482-5017   

                                 Web  
0  http://www.allsaintsdunwoody.org/  
1                                     
2                      